If you are running this on Google Colab, you need to uncomment (remove the `#`) and execute the following lines to install the Pyomo package, the solver, and some helper tools. If you are running this on Binder or elsewhere (e.g. your own computer) you can ignore this.

In [2]:
 !pip install pyomo==6.4.1
 !pip install "git+https://github.com/sjpfenninger/sen1511.git#egg=sen1511utils&subdirectory=sen1511utils"

 !wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
 !chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
 !bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local/
 !conda install -y -c conda-forge ipopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/sjpfenninger/sen1511.git to /tmp/pip-install-09kghjsn/sen1511utils_011c7ee081f44716874832a73a4f5b61
  Running command git clone -q https://github.com/sjpfenninger/sen1511.git /tmp/pip-install-09kghjsn/sen1511utils_011c7ee081f44716874832a73a4f5b61
  Resolved https://github.com/sjpfenninger/sen1511.git to commit ef590366c6e0122d689293d3ec3bc049549e8a15
     |████████████████████████████████| 11.3 MB 4.1 MB/s 
     |████████████████████████████████| 793 kB 53.3 MB/s 
     |████████████████████████████████| 59 kB 5.4 MB/s 
     |████████████████████████████████| 107 kB 50.2 MB/s 
     |████████████████████████████████| 382 kB 58.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.3 MB/s 
  Using cached jedi-0.18.1-py2.py3-none-any.whl (1.6 MB)
     |████████████████████████████████| 100 kB 9.7 MB/s 
     |████████████████████████████████| 15.7 MB 26.8 MB/s 


--2022-09-12 20:50:00--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104996770 (100M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.12.0-Linux-x86_64.sh.1’

Miniconda3-py37_4.1 100%[===================>] 100.13M  68.0MB/s    in 1.5s    

2022-09-12 20:50:02 (68.0 MB/s) - ‘Miniconda3-py37_4.12.0-Linux-x86_64.sh.1’ saved [104996770/104996770]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py37h06a4308_2
    - cffi==1.15.0=py

In [3]:
import pyomo.environ as pyo

from sen1511utils import summarise_results

# Assignment 4 - Nonlinear programming (NLP)

## NUMBER.a)

In [4]:
m = pyo.ConcreteModel(name = "Economic dispatch with quadratic cost functions")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.PG1 = pyo.Var(domain=pyo.NonNegativeReals)
m.PG2 = pyo.Var(domain=pyo.NonNegativeReals)
m.PG3 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

cost_0 = 10 +    1 * m.PG1 + 0.5 *  0.1 * m.PG1**2
cost_1 = 20 +  1.1 * m.PG2 + 0.5 * 0.05 * m.PG2**2
cost_2 = 20 + 1.25 * m.PG3 + 0.5 *  0.1 * m.PG3**2

m.cost = pyo.Objective(
    expr = cost_0 + cost_1 + cost_2,
    sense = pyo.minimize,
)

##
# 3. Constraints
##

# For the case of 80 MW demand (other cases: 5 MW, 30 MW)
m.demand = pyo.Constraint(expr = m.PG1 + m.PG2 + m.PG3 == 80)

m.pg1_max = pyo.Constraint(expr = m.PG1 <= 30)
m.pg2_max = pyo.Constraint(expr = m.PG2 <= 40)
m.pg3_max = pyo.Constraint(expr = m.PG3 <= 50)

# Solve the problem
solver = pyo.SolverFactory('ipopt')
solver.solve(m)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 3, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.14.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.09206938743591309}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [5]:
summarise_results(m)

<div class="alert alert-block alert-info">

💡 As usual, you could use Python to automate - and make it easier to extend this to a larger problem with many more power plants. For example, you might specify the cost function as an actual Python function and and sum it up with `sum()` in the objective:

```python
PG = [m.PG1, m.PG2, m.PG3]
C = [10, 20, 20]
a = [1, 1.1, 1.25]
b = [0.1, 0.05, 0.1]

def cost(i):
    return C[i] + a[i] * PG[i] + 0.5 * b[i] * PG[i]**2

m.cost = pyo.Objective(
    expr = sum(cost[i] for i in range(3)),
    sense = pyo.minimize,
)
```

</div>

## NUMBER.b)

In [9]:
m = pyo.ConcreteModel(name = "Economic dispatch with quadratic cost functions and elastic demand")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

# Generation
m.PG1 = pyo.Var(domain=pyo.NonNegativeReals)
m.PG2 = pyo.Var(domain=pyo.NonNegativeReals)
m.PG3 = pyo.Var(domain=pyo.NonNegativeReals)

# Demand
m.PD1 = pyo.Var(domain=pyo.NonNegativeReals)
m.PD2 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

PG = [m.PG1, m.PG2, m.PG3]
PD = [m.PD1, m.PD2]
C = [10, 20, 20]
a = [1, 1.1, 1.25]
b = [0.1, 0.05, 0.1]
ad = [55, 50]
bd = [-0.2, -0.1]

def cost(i):
    return C[i] + a[i] * PG[i] + 0.5 * b[i] * PG[i]**2

def util (j):
    return  ad[j]* PD[j]  +   bd[j] * PD[j]**2

m.SW = pyo.Objective(
    expr =  util (0) + util (1) - (cost(0) + cost(1) + cost(2)),
    sense = pyo.maximize,
)

##
# 3. Constraints
##

# For the case of 80 MW demand (other cases: 5 MW, 30 MW)
m.demand = pyo.Constraint(expr = m.PG1 + m.PG2 + m.PG3 == m.PD1 + m.PD2)

m.pg1_max = pyo.Constraint(expr = m.PG1 <= 30)
m.pg2_max = pyo.Constraint(expr = m.PG2 <= 40)
m.pg3_max = pyo.Constraint(expr = m.PG3 <= 50)
m.pd1_max = pyo.Constraint(expr = m.PD1 <= 300)
m.pd2_max = pyo.Constraint(expr = m.PD2 <= 350)


# Solve the problem
solver = pyo.SolverFactory('ipopt')
solver.solve(m)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 5, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.14.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.03775835037231445}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
summarise_results(m)